In [1]:
import pandas as pd

#load data from TSV

df1 = pd.read_csv('speech1.tsv', delimiter='\t')
df2 = pd.read_csv('conga3.tsv', delimiter='\t')

In [2]:
from mei import process_and_visualize

process_and_visualize(df1, save_mei=False, save_path='my_output.mei', display_df=False)
print("-" * 30)
process_and_visualize(df2, save_mei=False, save_path='my_output.mei', display_df=False)

------------------------------
